# `elec-00`: EEG preprocessing
This lab implements EEG preprocessing using bandpass filtering and independent component analysis (ICA). ICA-based preprocessing will allow us to identify and remove noise signals due to eye-blinks. We'll use the [MNE](https://mne.tools/stable/index.html) Python package for visualization and analysis. If you don't have MNE installed, run the following line in your conda environment: `conda install -c conda-forge mne-base`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load in raw data
Similar to fMRI, EEG data come in a variety of different file formats. We will use the Elektra Neuromag (fif) file format, typically associated with MEG data, as that is a commonly used data type in MNE. For loading different file formats into MNE, see this [tutorial](https://mne.tools/stable/auto_tutorials/io/index.html).

In [ ]:
from mne.io import read_raw

# Specify path to raw data
raw_fn = 'sub-01_task-audvis_raw.fif'

# Load raw data
data_raw = read_raw(raw_fn, preload=True, verbose=False)
print(data_raw)

### Visualizing the raw data
The `mne.io.Raw` class object is a very helpful data structure, containing the entire EEG recording and its corresponding metadata. The `.info` attribute allows us to easily inspect all of the metadata. For example, we can specify that channel `EEG 053` is a bad channel. Use `find_layout` from `mne.channels` and `plot_layout` from `mne.viz` to visualize the channel layout.

In [ ]:
# Inspect metadata
print(data_raw.info)

# Designate bad channel
data_raw.info['bads'] = ['EEG 053']

# Visualize channel layout:
from mne.channels import find_layout
from mne.viz import plot_layout


We can interactively visualize the raw EEG data using `Raw` object's `.plot()` method. You may need to specify a `%matplotlib` magic command for interactive plotting (e.g. `%matplotlib osx` for Mac).

In [ ]:
# Interactively plot raw data:
%matplotlib osx


We can also interactively visualize the standard (e.g. 10–20) channel layouts. Use `make_standard_montage` from `mne.channels` with the `'standard_1020'`) argument to create the standard montage. Use the montage's `.plot()` method (with `kind='3d'` or `kind='topomap'`) to visualize the channel layout.

In [ ]:
# Create standard 1020 channel locations:
from mne.channels import make_standard_montage


# Plot standard electrode locations:
%matplotlib osx


### Filtering

Filtering data can help remove high-frequency artifacts (e.g. EMG artifact) and low-frequency drifts, and notch filters at 50 Hz or 60 Hz help attenuate electrical line noise. Applying a high-pass filter at 0.1 Hz or 0.5 Hz to the continuous data is useful and recommended to minimize slow drifts. More conservative high-pass filters have the potential to [distort and bias EEG analysis](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4506207/). Use the `Raw` data's `.filter()` method to bandpass filter the EEG data between 0.5 and 40 Hz using the FIR method with zero phase. We'll use `pick_types` to specify only the EEG channels (e.g. excluding ocular channels, stimulus channels, etc). Name the output `data_filtered`.

In [ ]:
from mne import pick_types

# Select EEG channels (excluding other channels)
picks = pick_types(data_raw.info, meg=False, eeg=True, eog=False, stim=False)

# Apply bandpass filter:


### Independent component analysis (ICA)

There are many sources of artifact in EEG data, including head motion, muscle tension, recording drift, and channel pops. Perhaps the most common source of artifact is eyeblink. Oculomotor activity causes large deflections in the EEG recording (with decreasing magnitude in channels further from the eyes). There are a number of methods for removing eyeblinks, including amplitude rejection, signal space projection (SSP), and independent component analysis (ICA). ICA finds directions in the feature space corresponding to projections with high non-Gaussianity. We thus obtain a decomposition into independent components, and the artifact's contribution is typically localized in only a small number of components. These components must be correctly identified and removed. More complete information about the theory behind ICA and its application with MNE can be found [here](https://mne.tools/stable/auto_tutorials/preprocessing/plot_40_artifact_correction_ica.html). Initialize `ICA` from `mne.preprocessing` with `n_components` (the number PCs retained prior to ICA) set to 25, using `method='fastica'`. Use `pick_types` to specify the EEG channels, then use the ICA object's `.fit()` (with `decim=3`) to estimate the ICs from the EEG data.

In [ ]:
from mne.preprocessing import ICA

# Define parameters for ICA
n_components = 25
method = 'fastica'
decim = 3 

# Initialize MNE's ICA object:

# Fit ICA to filtered data:


We can visualize the scalp topography of each component to identify artifactual components. Use the fitted ICA object's `.plot_components()` method to plot the scalp topography of the ICs.

In [ ]:
# Plot ICA components:


MNE also has some nice functions for visually exploring features of each component. Try using the fitted ICA object's `.plot_properties()` method to visualize some potentially problematic ICs.

In [ ]:
# Visualize IC properties:


There's a much more efficient way to detect artifactual components with MNE. Instead, we will take snapshots of each instance of a blink and correlate these data with the ICA components in order to find those components most likely corresponding to eyeblinks.

To identify eyeblinks in the data, we will use the `create_eog_epochs` function, which conveniently looks for large events in the EOG (`ch_name='EOG 061'`; electro-oculography) channel. Use the resulting `.average()` method of the result EOG epochs object to average and plot the 

In [ ]:
from mne.preprocessing import create_eog_epochs

# Set parameters for creating epochs
reject = {'eeg': 2e-4}
picks = pick_types(data_filtered.info, meg=False, eeg=True, eog=True, stim=False)

# Create EOG epochs:

# Compute average eyeblink and plot:


Next we detect EOG related components using correlation. Detection is based on Pearson correlation between the filtered data and the filtered EOG channel. Thresholding is based on adaptive z-scoring. The above threshold components will be masked and the z-score will be recomputed until no supra-threshold component remains. Use the fitted ICA object's `.find_bads_eog()` method to obtain the EEG IC that corresponds to eyeblinks as well as the correlation scores. Use the fitted ICA object's `.plot_scores()` method to plot the scores.

In [ ]:
# Detect EOG components find_bads_eog:

# Plot correlation scores:


We can also inspect the ICA-based source timecourse within the time window of our EOG average. Use the fitted ICA object's `.plot_sources()` method with the average EOG epochs to visualize all of the sources during eyeblinks. One component should pop out!

In [ ]:
# Plot sources:


We can take a look at the properties of that component, now using the data epoched with respect to EOG events. Use `plot_properties` to visualize the the EOG epochs, specifically focusing on the EOG IC identified by `find_bads_eog`.

In [ ]:
# Plot problematic IC properties during eyeblink epochs:


Now let’s see how we would modify our signals if we removed this component from the data. Use the fitted ICA object's `.plot_overlay` function specifying the averaged EOG epoch and set `exclude` to the index of the IC corresponding to eyeblinks.

In [ ]:
# Plot overlay of before and after eyeblink IC removal:


To register this component as a bad one to be remove, use the `ica.exclude` attribute. This is a simple Python list; i.e. append the problematic IC index to `exclude` attribute of the fitted ICA object.

In [ ]:
# Add eyeblink IC index to exclude:


Note that nothing is yet removed from the raw data. To remove the effects of the rejected components, the `.apply()` method of the fitted ICA object must be called on the data. This will reconstruct the data without any ICs in the `exclude` list. We apply the ICA transformation to a copy of the original raw data.

In [ ]:
# Copy filtered raw data
data_ica = data_filtered.copy()

# Apply ICA decomposition and transformation:

# Save preprocessed data:


Finally, interactive re-visualize the preprocessed EEG data using the data's `.plot()` method.

In [ ]:
# Interactively plot preprocessed data:
%matplotlib osx
